## 1. Importação de Bibliotecas

In [ ]:
# Bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Bibliotecas importadas com sucesso!")

## 2. Ingestão de Dados

Vamos carregar dados históricos do Bitcoin usando a API do Yahoo Finance.

In [ ]:
from src.data_ingestion import CryptoDataIngestion

# Criar instância do ingestor
ingestion = CryptoDataIngestion()

# Buscar dados do Bitcoin (5 anos)
df_btc = ingestion.fetch_from_yahoo("BTC-USD", period="5y")

# Visualizar informações
info = ingestion.get_data_info(df_btc)
print(f"\n📊 Informações do Dataset:")
print(f"Registros: {info['rows']}")
print(f"Período: {info['date_range']['start']} a {info['date_range']['end']}")

# Preview dos dados
df_btc.head()

## 3. Análise Exploratória de Dados (EDA)

In [ ]:
# Estatísticas descritivas
print("📊 Estatísticas Descritivas:")
df_btc[['Close', 'Volume']].describe()

In [ ]:
# Gráfico de preços históricos
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df_btc['Date'], df_btc['Close'], linewidth=2, color='#2196F3')
ax.set_title('Histórico de Preços - Bitcoin (BTC-USD)', fontsize=16, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Preço (USD)', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💰 Preço Atual: ${df_btc['Close'].iloc[-1]:,.2f}")
print(f"📈 Máximo: ${df_btc['Close'].max():,.2f}")
print(f"📉 Mínimo: ${df_btc['Close'].min():,.2f}")

In [ ]:
# Distribuição de retornos
df_btc['Daily_Return'] = df_btc['Close'].pct_change() * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(df_btc['Daily_Return'].dropna(), bins=50, color='skyblue', edgecolor='black')
axes[0].set_title('Distribuição de Retornos Diários', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Retorno (%)', fontsize=12)
axes[0].set_ylabel('Frequência', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot(df_btc['Daily_Return'].dropna())
axes[1].set_title('Boxplot de Retornos Diários', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Retorno (%)', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Volatilidade (desvio padrão): {df_btc['Daily_Return'].std():.2f}%")

## 4. Processamento com Apache Spark

Utilizando PySpark para calcular médias móveis e indicadores técnicos.

In [ ]:
from src.spark_processor import SparkDataProcessor

# Inicializar Spark
processor = SparkDataProcessor()

# Calcular médias móveis
print("⚡ Calculando médias móveis com Spark...")
df_with_ma = processor.calculate_moving_averages(
    df_btc,
    windows=[7, 30, 90, 200]
)

# Calcular indicadores técnicos
print("⚡ Calculando indicadores técnicos...")
df_processed = processor.calculate_technical_indicators(df_with_ma)

print("\n✅ Processamento concluído!")
print(f"Colunas adicionadas: {[col for col in df_processed.columns if col not in df_btc.columns]}")

df_processed.tail()

In [ ]:
# Visualizar preços com médias móveis
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df_processed['Date'], df_processed['Close'], label='Preço', linewidth=2, color='#2196F3')
ax.plot(df_processed['Date'], df_processed['MA_7'], label='MA 7d', linewidth=1.5, linestyle='--', color='#FFC107')
ax.plot(df_processed['Date'], df_processed['MA_30'], label='MA 30d', linewidth=1.5, linestyle='--', color='#4CAF50')
ax.plot(df_processed['Date'], df_processed['MA_90'], label='MA 90d', linewidth=1.5, linestyle='--', color='#9C27B0')

ax.set_title('Preço com Médias Móveis', fontsize=16, fontweight='bold')
ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Preço (USD)', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Modelagem com Prophet (Séries Temporais)

Usando Prophet do Facebook/Meta para prever tendências futuras.

In [ ]:
from models.prophet_model import CryptoProphetModel, PROPHET_AVAILABLE

if PROPHET_AVAILABLE:
    # Criar modelo
    prophet_model = CryptoProphetModel()
    
    # Preparar dados
    df_prophet = prophet_model.prepare_data(df_btc)
    
    # Dividir em treino e teste (80/20)
    split_idx = int(len(df_prophet) * 0.8)
    df_train = df_prophet[:split_idx]
    df_test = df_prophet[split_idx:]
    
    print(f"📊 Treino: {len(df_train)} registros")
    print(f"📊 Teste: {len(df_test)} registros")
    
    # Treinar modelo
    print("\n🔮 Treinando Prophet...")
    prophet_model.train(df_train, changepoint_prior_scale=0.05, seasonality_mode='multiplicative')
    
    # Fazer previsões
    print("🔮 Fazendo previsões...")
    forecast = prophet_model.predict(periods=30)
    
    # Avaliar
    metrics = prophet_model.evaluate(df_test)
    print(f"\n📊 Métricas de Avaliação:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")
    
    # Análise de tendência
    trend_analysis = prophet_model.get_trend_analysis()
    print(f"\n📈 Análise de Tendência:")
    print(f"Direção: {trend_analysis['direction']}")
    print(f"Variação: {trend_analysis['trend_change_pct']:.2f}%")
    
    # Salvar modelo
    prophet_model.save_model("models/prophet_btc_model.pkl")
    print("\n✅ Modelo Prophet salvo!")
else:
    print("⚠️ Prophet não está disponível. Instale com: pip install prophet")

In [ ]:
# Visualizar previsões Prophet
if PROPHET_AVAILABLE and 'forecast' in locals():
    forecast_summary = prophet_model.get_forecast_summary(30)
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Dados históricos
    ax.plot(df_btc['Date'], df_btc['Close'], label='Histórico', linewidth=2, color='#2196F3')
    
    # Previsões
    ax.plot(forecast_summary['Date'], forecast_summary['Predicted'], 
            label='Previsão', linewidth=2, linestyle='--', color='#FF5722', marker='o')
    
    # Intervalos de confiança
    ax.fill_between(forecast_summary['Date'], 
                     forecast_summary['Lower_Bound'], 
                     forecast_summary['Upper_Bound'], 
                     alpha=0.3, color='#FF5722')
    
    ax.set_title('Previsão Prophet - Próximos 30 Dias', fontsize=16, fontweight='bold')
    ax.set_xlabel('Data', fontsize=12)
    ax.set_ylabel('Preço (USD)', fontsize=12)
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Previsões para os próximos 7 dias:")
    print(forecast_summary.head(7))

## 6. Modelagem com scikit-learn

Usando Regressão Linear para previsão de preços.

In [ ]:
from models.sklearn_model import CryptoMLModel, SKLEARN_AVAILABLE
from sklearn.model_selection import train_test_split

if SKLEARN_AVAILABLE:
    # Criar modelo
    ml_model = CryptoMLModel(model_type='linear_regression')
    
    # Preparar features
    print("🤖 Preparando features...")
    X, y = ml_model.prepare_features(df_btc)
    
    print(f"Features: {len(ml_model.feature_names)}")
    print(f"Amostras: {len(X)}")
    
    # Dividir dados
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )
    
    # Treinar
    print("\n🤖 Treinando modelo...")
    ml_model.train(X_train, y_train)
    
    # Avaliar
    metrics = ml_model.evaluate(X_test, y_test)
    print(f"\n📊 Métricas de Avaliação:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")
    
    # Prever próximos dias
    forecast_ml = ml_model.predict_next_days(df_btc, days=7)
    print(f"\n📈 Previsões para próximos 7 dias:")
    print(forecast_ml)
    
    # Salvar modelo
    ml_model.save_model("models/sklearn_btc_model.pkl")
    print("\n✅ Modelo scikit-learn salvo!")
else:
    print("⚠️ scikit-learn não está disponível")

In [ ]:
# Visualizar performance do modelo
if SKLEARN_AVAILABLE and 'ml_model' in locals():
    y_pred = ml_model.predict(X_test)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Valores reais vs previstos
    axes[0].scatter(y_test, y_pred, alpha=0.5)
    axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[0].set_xlabel('Valores Reais', fontsize=12)
    axes[0].set_ylabel('Valores Previstos', fontsize=12)
    axes[0].set_title('Reais vs Previstos', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Resíduos
    residuals = y_test - y_pred
    axes[1].scatter(y_pred, residuals, alpha=0.5)
    axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
    axes[1].set_xlabel('Valores Previstos', fontsize=12)
    axes[1].set_ylabel('Resíduos', fontsize=12)
    axes[1].set_title('Análise de Resíduos', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Visualizações Interativas com Plotly

In [ ]:
from visualizations.plotly_charts import CryptoVisualizer, PLOTLY_AVAILABLE

if PLOTLY_AVAILABLE:
    visualizer = CryptoVisualizer()
    
    # Gráfico de preços com MAs
    fig_price = visualizer.plot_price_history(
        df_processed,
        show_ma=True,
        ma_windows=[7, 30, 90]
    )
    fig_price.show()
    
    # Salvar
    visualizer.save_figure(fig_price, 'btc_price_analysis.html')
    print("✅ Gráfico salvo em: visualizations/btc_price_analysis.html")
else:
    print("⚠️ Plotly não está disponível")

In [ ]:
# Dashboard completo
if PLOTLY_AVAILABLE:
    fig_dashboard = visualizer.plot_dashboard(df_processed)
    fig_dashboard.show()
    
    visualizer.save_figure(fig_dashboard, 'btc_dashboard.html')
    print("✅ Dashboard salvo em: visualizations/btc_dashboard.html")

## 8. Conclusões e Insights

### Resumo da Análise:

1. **Ingestão de Dados**: ✅ Dados históricos carregados do Yahoo Finance
2. **Processamento Spark**: ✅ Médias móveis e indicadores calculados
3. **Prophet**: ✅ Modelo treinado para previsão de tendências
4. **scikit-learn**: ✅ Regressão Linear para previsão de preços
5. **Visualizações**: ✅ Gráficos interativos com Plotly

### Próximos Passos:

- Integrar dados de outras equipes
- Testar modelos mais avançados (XGBoost, LSTM)
- Implementar sistema de alertas
- Deploy em produção (Databricks)

In [ ]:
# Salvar dados processados
output_path = Path("data/processed/btc_processed.csv")
df_processed.to_csv(output_path, index=False)
print(f"\n✅ Dados processados salvos em: {output_path}")

# Encerrar Spark
if 'processor' in locals():
    processor.stop()
    print("✅ Sessão Spark encerrada")

print("\n🎉 Análise concluída com sucesso!")